In [1]:
from jet_ML_tools import *
from data_import import data_import

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

Using Theano backend.
Using gpu device 0: GeForce GTX 660M (CNMeM is enabled with initial size: 15.0% of memory, cuDNN 5105)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
# specify the file inputs
n_files = 1
n_ev_per_file = 10000
s_range = range(1,2)

# read in the data
nevents = n_files * n_ev_per_file
data = data_import(data_type='jetimage', seed_range=s_range, path='../images/',nevents=nevents, img_size=33, channels=[0])

# read in the labels and split the data by quark and gluon
g_data, q_data = data[:10000], data[10000:]
labels = to_categorical(np.concatenate((np.zeros(nevents),np.ones(nevents))), num_cat = 2)

In [3]:
# Adapted from: github.com/derylucio/weaksupervision

from keras.optimizers import Adam

# Weak supervision loss function
def weak_loss_function(ytrue, ypred):
    return K.square(K.sum(ypred)/ypred.shape[0] - K.sum(ytrue)/ypred.shape[0])

# Generates batches for training with keras
def weak_data_generator(samples, output):
    while True:
        for i in range(len(samples)):
            yield samples[i], output[i]

# Train a network with weak supervision
# UNDER CONSTRUCTION - EMM
def trainCNN(samples, targets, layersize = 30, nb_epoch = 10, learning_rate = 0.001, val_frac = 0.1, weak = True):
    
    listX_train, listX_val = [], []
    listf_train, listf_val = [], []
    
    # split the data and fractions into training and validation sets
    for X,f in zip(samples,targets):
        X_train, f_train, X_val, f_val = data_split(X, f, val_frac = val_frac, test_frac=0.0)
        listX_train.append(X_train)
        listf_train.append(f_train)
        listX_val.append(X_val)
        listf_val.append(f_val)
    
    trainsize = sum([X.shape[0] for X in listX_train])
        
    # construct the CNN
    hps = {'batch_size': 128, 'img_size': 33, 'nb_conv': [8,4,4], 'nb_filters': [64, 64, 64],
           'nb_neurons': 128, 'nb_pool': [2, 2, 2], 'dropout': [.25, .5, .5, .5],
           'nb_channels': 1, 'patience': 3, 'out_dim' : targets[0].shape[1]}
    
    CNN_model = conv_net_construct(hps, compiled = False)
    earlystopper = EarlyStopping(monitor="val_loss", patience= hps['patience'])
    
    if weak:
        CNN_model.compile(loss = weak_loss_function, optimizer=Adam(lr=learning_rate), metrics = ['accuracy']) 
    else:
        CNN_model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=learning_rate), metrics = ['accuracy'])
        
    CNN_model.summary()
    CNN_model.fit_generator(generator = weak_data_generator(listX_train, listf_train), samples_per_epoch = trainsize,
                             nb_epoch = nb_epoch, validation_data = weak_data_generator(listX_val, listf_val), 
                             nb_val_samples = len(listX_val), callbacks = [earlystopper])

    return CNN_model

In [4]:
trainCNN([data], [labels], weak = False)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 64, 26, 26)    4160        convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 64, 26, 26)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 64, 13, 13)    0           activation_1[0][0]               
____________________________________________________________________________________________________
spatialdropout2d_1 (SpatialDropo (None, 64, 13, 13)    0           maxpooling2d_1[0][0]             
___________________________________________________________________________________________

MemoryError: Error allocating 3115008000 bytes of device memory (CNMEM_STATUS_OUT_OF_MEMORY).
Apply node that caused the error: GpuAllocEmpty(Shape_i{0}.0, Shape_i{0}.0, Elemwise{Composite{((((i0 + i1) - i2) // i3) + i3)}}[(0, 2)].0, Elemwise{Composite{((((i0 + i1) - i2) // i3) + i3)}}[(0, 2)].0)
Toposort index: 92
Inputs types: [TensorType(int64, scalar), TensorType(int64, scalar), TensorType(int64, scalar), TensorType(int64, scalar)]
Inputs shapes: [(), (), (), ()]
Inputs strides: [(), (), (), ()]
Inputs values: [array(18000), array(64), array(26), array(26)]
Outputs clients: [[GpuDnnConv{algo='small', inplace=True}(GpuContiguous.0, GpuContiguous.0, GpuAllocEmpty.0, GpuDnnConvDesc{border_mode='valid', subsample=(1, 1), conv_mode='conv', precision='float32'}.0, Constant{1.0}, Constant{0.0})]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If t